In [14]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [15]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_spaceNet.ipynb


In [16]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [17]:
from scipy import spatial
from sklearn.decomposition import PCA

In [18]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [19]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [20]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [21]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [22]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1312.98,-0.01,0.02,0.02,0.00,0.00,0.02,0.03,-0.00,0.01,...,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
3,-0.01,939.22,0.00,-0.02,0.02,-0.03,0.00,0.01,-0.00,-0.00,...,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00
4,0.02,0.00,721.97,0.03,0.01,-0.00,0.01,-0.01,0.01,0.00,...,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
5,0.02,-0.02,0.03,533.56,-0.02,0.02,-0.00,-0.01,0.00,0.00,...,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
6,0.00,0.02,0.01,-0.02,366.11,-0.00,-0.00,-0.00,-0.01,0.00,...,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
7,0.00,-0.03,-0.00,0.02,-0.00,232.75,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.02,0.00,0.01,-0.00,-0.00,-0.00,152.25,0.00,-0.01,0.00,...,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
9,0.03,0.01,-0.01,-0.01,-0.00,0.00,0.00,91.32,-0.00,-0.00,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
10,-0.00,-0.00,0.01,0.00,-0.01,0.00,-0.01,-0.00,58.72,0.00,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00
11,0.01,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,37.27,...,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00


In [23]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00002,0.00003,0.00000,0.00001,0.00004,0.00010,-0.00001,0.00003,...,0.00001,0.00001,-0.00002,0.00007,0.00005,-0.00000,0.00002,-0.00001,0.00003,-0.00002
3,-0.00001,1.00000,0.00000,-0.00002,0.00003,-0.00006,0.00001,0.00003,-0.00002,-0.00002,...,-0.00001,0.00004,0.00000,0.00003,0.00003,-0.00001,0.00003,-0.00004,-0.00004,-0.00001
4,0.00002,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00007,0.00000,...,-0.00004,0.00001,0.00003,-0.00004,0.00000,0.00001,0.00000,-0.00002,0.00002,0.00000
5,0.00003,-0.00002,0.00004,1.00000,-0.00004,0.00005,-0.00000,-0.00003,0.00001,0.00001,...,-0.00002,-0.00002,0.00001,-0.00001,0.00004,0.00000,0.00001,-0.00002,0.00005,0.00003
6,0.00000,0.00003,0.00002,-0.00004,1.00000,-0.00000,-0.00000,-0.00000,-0.00004,0.00004,...,0.00002,-0.00001,0.00002,0.00000,0.00002,0.00004,-0.00002,0.00002,-0.00004,-0.00001
7,0.00001,-0.00006,-0.00000,0.00005,-0.00000,1.00000,-0.00000,0.00000,0.00003,-0.00002,...,-0.00004,0.00002,-0.00003,0.00008,-0.00003,0.00005,0.00001,-0.00005,0.00000,-0.00003
8,0.00004,0.00001,0.00003,-0.00000,-0.00000,-0.00000,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00003,-0.00001,0.00000,-0.00000,0.00000,-0.00002,-0.00007,-0.00003,-0.00000
9,0.00010,0.00003,-0.00004,-0.00003,-0.00000,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,-0.00000,-0.00001,0.00005,-0.00002,0.00003,0.00001,0.00002,-0.00003,-0.00006
10,-0.00001,-0.00002,0.00007,0.00001,-0.00004,0.00003,-0.00005,-0.00003,1.00000,0.00006,...,0.00001,-0.00000,0.00000,-0.00002,0.00001,0.00003,-0.00003,0.00002,0.00001,0.00002
11,0.00003,-0.00002,0.00000,0.00001,0.00004,-0.00002,0.00005,-0.00002,0.00006,1.00000,...,0.00001,0.00001,0.00005,-0.00001,0.00004,0.00006,-0.00004,-0.00001,-0.00006,0.00000


In [24]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.11266701747429002

In [25]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.52194133e+04 3.83879437e+04 2.25756139e+04 1.22829571e+04
 5.76705231e+03 2.32631820e+03 9.91264246e+02 3.55899687e+02
 1.46455134e+02 5.87408392e+01 2.49672268e+01 1.09823494e+01
 4.84749272e+00 1.98181742e+00 7.97704500e-01 2.83760574e-01
 9.80578472e-02 2.56615368e-02 7.24689877e-03 1.78238959e-03
 7.42788598e-04 3.74828519e-04 2.49502534e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997067,-0.063800,-0.033392,-0.020400,-0.012629,-0.007624,-0.004882,-0.002882,-0.001867,-0.001177,...,-0.000219,-0.000140,-0.000085,-0.000048,-0.000025,-0.000014,-0.000007,-0.000005,-0.000003,-0.000003
1,0.059398,0.993230,-0.088789,-0.037939,-0.020458,-0.011746,-0.007365,-0.004329,-0.002783,-0.001761,...,-0.000328,-0.000207,-0.000126,-0.000073,-0.000038,-0.000021,-0.000010,-0.000007,-0.000005,-0.000004
2,0.036081,0.082202,0.991512,-0.084777,-0.034030,-0.017757,-0.010829,-0.006325,-0.003996,-0.002540,...,-0.000477,-0.000301,-0.000179,-0.000110,-0.000056,-0.000030,-0.000015,-0.000010,-0.000007,-0.000005
3,0.024275,0.041344,0.077272,0.992583,-0.072454,-0.028926,-0.016584,-0.009396,-0.005916,-0.003729,...,-0.000695,-0.000444,-0.000265,-0.000159,-0.000080,-0.000044,-0.000022,-0.000015,-0.000010,-0.000007
4,0.015990,0.024163,0.035585,0.065879,0.994229,-0.062270,-0.028563,-0.014991,-0.009273,-0.005765,...,-0.001064,-0.000682,-0.000407,-0.000244,-0.000124,-0.000067,-0.000034,-0.000021,-0.000017,-0.000012
5,0.010034,0.014487,0.019639,0.029156,0.056495,0.994625,-0.069530,-0.027253,-0.015710,-0.009579,...,-0.001747,-0.001108,-0.000670,-0.000390,-0.000206,-0.000109,-0.000054,-0.000037,-0.000026,-0.000020
6,0.006903,0.009784,0.012940,0.018197,0.029250,0.064482,0.994673,-0.060772,-0.028728,-0.016327,...,-0.002865,-0.001822,-0.001099,-0.000652,-0.000337,-0.000182,-0.000091,-0.000061,-0.000044,-0.000032
7,0.004221,0.005957,0.007828,0.010716,0.016113,0.027891,0.055065,0.994244,-0.074532,-0.032159,...,-0.004961,-0.003159,-0.001897,-0.001120,-0.000584,-0.000313,-0.000155,-0.000100,-0.000076,-0.000057
8,0.002921,0.004096,0.005315,0.007254,0.010736,0.017471,0.029260,0.068388,0.992929,-0.077702,...,-0.008387,-0.005312,-0.003179,-0.001890,-0.000973,-0.000525,-0.000263,-0.000169,-0.000125,-0.000092
9,0.001952,0.002743,0.003565,0.004832,0.007074,0.011310,0.017834,0.033311,0.069844,0.991422,...,-0.014352,-0.008962,-0.005319,-0.003163,-0.001623,-0.000875,-0.000441,-0.000284,-0.000213,-0.000155


In [26]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.002932935039209994,
 0.006769784051447125,
 0.008488043946489943,
 0.007416769180842664,
 0.0057707037105731995,
 0.005374802144829216,
 0.005327010736147275,
 0.0057562664291535,
 0.00707077779572185,
 0.008578269295254848,
 0.010956573103501288,
 0.013449909007451244,
 0.015020533184660123,
 0.01644936036498823,
 0.017633751283453014,
 0.018969655976850852,
 0.01936353528390322,
 0.02208141191343327,
 0.03022232205070552,
 0.06676541986937745,
 0.15826214450790643,
 0.31052553651075243,
 0.29518968822901814]